In [136]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [137]:
df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])

rating 값이 3 이상인 데이터만 뽑으면 user 한 명이 아예 사라져 행렬계산 시 오류나서 우선 rating 값이 있으면 모두 1로 binarize 했습니다.

In [138]:
#df_ratings = df_ratings[df_ratings['Rating'] > 2]
df_ratings['Rating'] = 1
df_ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,1,978300760
1,1,661,1,978302109
2,1,914,1,978301968
3,1,3408,1,978300275
4,1,2355,1,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,1,956704887
1000206,6040,562,1,956704746
1000207,6040,1096,1,956715648


In [140]:
df_user_movie_ratings = df_ratings.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
df_user_movie_ratings

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


k=13으로 하여 svd 실시

In [195]:
U, sigma, Vt = svds(df_user_movie_ratings, k=13)

In [144]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(6040, 13)
(13,)
(13, 3706)


In [145]:
sigma = np.diag(sigma)

In [146]:
sigma.shape

(13, 13)

In [147]:
sigma

array([[ 69.57219577,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,  73.83835647,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,  76.65207391,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,  84.98229881,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         86.69480598,   0.        ,   0.        ,   0.        ,
          0.

In [150]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns=df_user_movie_ratings.columns)
df_svd_preds.index += 1
df_svd_preds

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,0.396842,0.155316,0.018502,0.007772,0.020813,-0.053513,0.026782,0.042938,-0.025168,0.024798,...,0.003434,0.003678,0.021732,-0.006770,-0.014508,0.071879,0.023873,-0.002562,0.001305,0.006236
2,0.273231,0.069645,0.093674,0.027318,0.048781,0.342619,0.150283,-0.001943,0.040022,0.321331,...,-0.005217,0.000154,-0.006442,0.019375,0.009389,0.119582,-0.004369,0.002989,-0.002332,0.079954
3,0.337971,0.032915,0.041522,-0.020913,0.010650,0.084830,0.028098,0.000940,-0.003630,0.127603,...,-0.001342,-0.002220,0.006363,-0.000190,-0.004717,0.104877,-0.009108,-0.007206,-0.005873,0.000818
4,0.110254,0.014194,-0.014561,-0.029501,-0.031577,0.057768,-0.034764,-0.001455,-0.004304,0.062542,...,-0.004949,-0.001529,0.001809,-0.001713,-0.002943,0.023989,-0.008305,-0.002425,-0.004062,0.003163
5,0.392356,0.053622,-0.100204,0.046611,-0.067279,0.424864,0.006221,0.000161,-0.004256,0.130770,...,0.033092,-0.001272,-0.011185,0.002203,-0.001126,0.093119,0.163016,0.015605,0.021646,0.104603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.542248,0.490684,-0.030712,0.193050,-0.001925,0.513978,0.344464,0.011649,0.021183,0.343139,...,0.050861,0.001228,-0.013706,-0.006545,0.046267,-0.047336,0.248745,0.062689,0.057510,0.168754
6037,0.442534,-0.020899,0.001910,-0.027137,-0.023348,0.178501,-0.040741,-0.014148,-0.032614,-0.055703,...,0.017045,-0.000660,-0.009325,-0.006397,0.025235,0.006051,0.006040,0.011974,-0.002730,0.019177
6038,0.107631,0.007352,-0.000471,-0.002487,-0.003589,-0.003584,0.018953,0.000228,-0.010435,0.008811,...,-0.001816,-0.000765,0.000740,-0.006287,-0.001751,0.002870,-0.007900,-0.003051,-0.000563,-0.010999
6039,0.507285,0.136193,0.049079,-0.008332,0.037914,-0.107933,0.120869,0.034470,-0.025601,0.072783,...,0.013905,0.006198,0.020854,-0.008862,0.021748,0.013835,-0.039789,0.000030,0.003592,-0.044997


In [185]:
x = df_ratings.copy()
x_train, x_test = train_test_split(x, test_size=0.05)

적절한 값 기준으로 하여 봤다고 예측, recall 계산

In [186]:
# 0.5 기준
cnt = 0
for i in x_test.index:
    if (df_svd_preds[x_test['MovieID'][i]][x_test['UserID'][i]] >= 0.5):
        cnt += 1

cnt/len(x_test)

0.31787006858491135

In [187]:
# 해당 user의 평균 기준
cnt = 0
for i in x_test.index:
    if (df_svd_preds[x_test['MovieID'][i]][x_test['UserID'][i]] >= np.mean(df_svd_preds.loc[x_test['UserID'][i]])):
        cnt += 1

cnt/len(x_test)

0.9042810581672032

In [188]:
# 해당 user의 중앙값 기준
cnt = 0
for i in x_test.index:
    if (df_svd_preds[x_test['MovieID'][i]][x_test['UserID'][i]] >= np.median(df_svd_preds.loc[x_test['UserID'][i]])):
        cnt += 1

cnt/len(x_test)

0.9671472276099258

k=20으로 하여 svd 실시

In [191]:
U2, sigma2, Vt2 = svds(df_user_movie_ratings, k=20)
sigma2 = np.diag(sigma2)

svd_user_predicted_ratings2 = np.dot(np.dot(U2, sigma2), Vt2)
df_svd_preds2 = pd.DataFrame(svd_user_predicted_ratings2, columns=df_user_movie_ratings.columns)
df_svd_preds2.index += 1

x2 = df_ratings.copy()
x2_train, x2_test = train_test_split(x2, test_size=0.05)

적절한 값 기준으로 하여 봤다고 예측, recall 계산

In [192]:
# 0.5 기준
cnt = 0
for i in x2_test.index:
    if (df_svd_preds2[x2_test['MovieID'][i]][x2_test['UserID'][i]] >= 0.5):
        cnt += 1

cnt/len(x2_test)

0.35252244506208635

In [193]:
# 해당 user의 평균 기준
cnt = 0
for i in x2_test.index:
    if (df_svd_preds2[x2_test['MovieID'][i]][x2_test['UserID'][i]] >= np.mean(df_svd_preds2.loc[x2_test['UserID'][i]])):
        cnt += 1

cnt/len(x2_test)

0.9140189158385156

In [194]:
# 해당 user의 중앙값 기준
cnt = 0
for i in x2_test.index:
    if (df_svd_preds2[x2_test['MovieID'][i]][x2_test['UserID'][i]] >= np.median(df_svd_preds2.loc[x2_test['UserID'][i]])):
        cnt += 1

cnt/len(x2_test)

0.9731858991021975

k가 클수록 recall 높음, 기준을 평균/중앙값으로 했더니 recall 크게 증가(90% 이상)

In [204]:
print(np.mean(df_svd_preds.loc[520]))
print(np.median(df_svd_preds.loc[520]))

0.09855228856881962
0.043140503246548834


평균/중앙값 대부분 0.1 이하로 매우 작음 -> 0에 가까운데 봤다고 하는 기준으로 삼아도 되는지 고민
(precision 계산해보고 결정? 영화 추천 시 recall만 높으면 상관 없는지?)